# Introduction to SSA in Scalpel

## What is the Static Single Assignment (SSA)

**SSA** is a means of structing the IR (intermediate representation) in compiling design, where requires each variable is defined exactly once. It can be used to generate use-def chain by renaming each variable assignment with different names, which can track precisely how data is flowing in the program. Some Algorithms can be improved by the application of SSA like dead code emiliation, value numbering and constant propagation. 
Generally, a variable may be assigned more than once in a program which will result in many instances. Taking the following as an example. On the left, the first line of y seems useless since the x only uses the second assignment of y. However, a program needs to find the reaching definition to achieve this. On the right example which is an SSA form where all of them are immediate:

$$y = 1 $\;\;\;\;\;\;$  ->$\;\;\;\;\;\;$y_1 = 1$$

$$y = 2 $\;\;\;\;\;\;$  ->$\;\;\;\;\;\;$y_2 = 2$$

$$x = y $\;\;\;\;\;\;$  ->$\;\;\;\;\;\;$x = y_2$$

## Let's install Scalpel first

Use the command in your virtual environment to install Scalpel.
```console
python -m pip install .
```

### Import essential libraries

In [1]:
import os
import sys
import ast
import astor
import unittest

### Import MNode and SSA API

In [2]:
from scalpel.core.mnode import MNode
from scalpel.SSA.const import SSA

### Let's have an example code

In [3]:
code_str = """
b = 10
if b>0:
    a = a+b
else:
    a = 10
print(a)
"""

In this example, the variable **a** has two possible values in the if-else statement. Without SSA, the value of a could only be one result. However, by applying **phi function** in SSA, an initial assignment to a has been added with blank value so that instance of a will have two corresponding values. Let's check how this happending.

### Generate CFGs for given source file

In [4]:
mnode = MNode("local") # create file name for given source code
mnode.source = code_str
mnode.gen_ast() # build abstract syntax tree
cfg = mnode.gen_cfg() 

In the MNode class, there are some attributes that has to be defined, where 'local' is the file name and source is the given source code. The function **mnode.gen_ast()** is parsing the source code, which is to build an AST tree for source code first. Then using cfg building method to generate a CFG for the function.

### Compute SSA

In [5]:
m_ssa = SSA()
ssa_results, const_dict = m_ssa.compute_SSA(cfg)

**compute_SSA** is the core function inside SSA class. The constant value and alias pairs are generated by computing cfg. First, it will compute the dominance frontier then using that to place phi node which is the two possible value of a in the example. Through creating and visiting all the blocks, finally renaming the variables so that each of them will have a unique name. Please note the function m_ssa.compute_SSA() will return two dictionaries.

### Building SSA graph

In [6]:
for block_id, stmt_res in ssa_results.items():
    print("These are the results for block ".format(block_id))
    print(stmt_res)

These are the results for block 
[{}, {'b': {0}}]
These are the results for block 
[{'a': {0}, 'b': {0}}]
These are the results for block 
[{}]
These are the results for block 
[{'print': set(), 'a': {0, 1}}]


The key in the first dictionary is the **block numbers** in given CFG, the value is the **statement result** of the corresponding block. Here is the output of the results. For the first block, the variable a has not been assignment so the result is the blank while the variable b is 0. In the second block when the if statement is true, the variable a will have a value with 0 and b remains the value. After computing SSA, the last block is after the if statement, the variable a takes two possible values.

The second dictionary is the global constant values for the numbered identifiers. There are many types of constant values.

In [7]:
for name, value in const_dict.items():
    print(name, value)

('b', 0) <ast.Constant object at 0x7fef60113e80>
('a', 0) <ast.BinOp object at 0x7fef60123b80>
('a', 1) <ast.Constant object at 0x7fef60123a90>


In [8]:
print(ssa_results)

{1: [{}, {'b': {0}}], 2: [{'a': {0}, 'b': {0}}], 4: [{}], 3: [{'print': set(), 'a': {0, 1}}]}


### Alias analysis
The alias pairs can be implemented, since the const_dict stores all the constant values of variables. If the assigned value is an ast.Name type, the alias pairs can be found as follow.

In [9]:
alias_name_pairs = []
for name, value in const_dict.items():
    if isinstance(value, ast.Name):
        alias_name_pairs.append((name, value.id))